In [1]:
from Tirto import Tirto #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [3]:
obj = Tirto()
end_date = date(2018, 8, 8)
start_date = date(2018, 1, 29)
init_page = 2

In [4]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], init_page, single_date.strftime("%Y-%m-%d"))
    init_page = 1

2018-01-29
page  2
https://tirto.id/indeks/2?date=2018-01-29
https://tirto.id/ketua-mpr-zulkifli-akan-temui-jokowi-tolak-rencana-impor-beras-cDZb
Insert berita  Ketua MPR Zulkifli akan Temui Jokowi Tolak Rencana Impor Beras
salah2
https://tirto.id/sekjen-pdip-puan-tunggu-arahan-jokowi-soal-rangkap-jabatan-menteri-cDZf
Insert berita  Sekjen PDIP: Puan Tunggu Arahan Jokowi Soal Rangkap Jabatan Menteri
salah2
https://tirto.id/new-honda-cbr150r-hadir-dengan-dua-warna-baru-cDZe
Insert berita  New Honda CBR150R Hadir dengan Dua Warna Baru
salah2
https://tirto.id/hasil-psps-vs-borneo-fc-di-piala-presiden-skor-akhir-0-3-cDZi
Insert berita  Hasil PSPS vs Borneo FC di Piala Presiden Skor Akhir 0-3
salah2
https://tirto.id/mendagri-pj-gubernur-papua-saat-pilkada-2018-bisa-dari-tni-polri-cDY9
Insert berita  Mendagri: Pj Gubernur Papua Saat Pilkada 2018 Bisa dari TNI/Polri
salah2
https://tirto.id/kabar-mobilisasi-becak-ke-dki-dan-inkonsistensi-ucapan-anies-sandi-cDYY
Insert berita  Kabar Mobilisasi 

IndexError: list index out of range